In [3]:
class Location:
    def __init__(self, description, exits):
        self.description = description
        self.exits = exits
        self.items = []
        self.blocks = []

class Item:
    def __init__(self, name, description, can_take=True):
        self.name = name
        self.description = description
        self.can_take = can_take

class Block:
    def __init__(self, name, solved=False):
        self.name = name
        self.solved = solved

class Game:
    def __init__(self):
        self.locations = {}
        self.current_location = None
        self.inventory = []
        self.game_over = False

    def setup_locations(self):
        staircase = Location("You are at the top of the staircase leading down to the entrance hall.", {'down': 'entrance hall'})
        entrance_hall = Location("You are in the entrance hall of the old castle.", {'north': 'library'})
        library = Location("You are in the grand library, filled with ancient books.", {'south': 'entrance hall', 'east': 'royal chamber'})
        royal_chamber = Location("This is the royal chamber. The crown must be here somewhere!", {'west': 'library'})

        entrance_hall.items.append(Item("torch", "An old but still usable torch."))
        library.items.append(Item("book of lore", "This book seems to contain useful information about the castle."))
        royal_chamber.items.append(Item("crown", "The ancient, priceless crown of the kingdom.", can_take=False))

        royal_chamber.blocks.append(Block("darkness"))

        self.locations = {
            "staircase": staircase,
            "entrance hall": entrance_hall,
            "library": library,
            "royal chamber": royal_chamber
        }
        self.current_location = staircase

    def start_game(self):
        self.setup_locations()
        while not self.game_over:
            self.describe_location()
            command = input("What do you want to do? ").lower()
            self.handle_command(command)

    def describe_location(self):
        print(self.current_location.description)
        for item in self.current_location.items:
            print(f"You see a {item.name}: {item.description}")
        if self.current_location.blocks:
            for block in self.current_location.blocks:
                if not block.solved and block.name == "darkness" and "torch" not in self.inventory:
                    print("It's too dark to see anything. You might need some light to proceed.")

    def handle_command(self, command):
        if command.startswith("go "):
            direction = command.split()[1]
            if direction in self.current_location.exits:
                self.current_location = self.locations[self.current_location.exits[direction]]
                print(f"Moving {direction}")
            else:
                print("You can't go that way.")
        elif command.startswith("take "):
            item_name = command.split()[1]
            item = next((item for item in self.current_location.items if item.name == item_name and item.can_take), None)
            if item:
                self.inventory.append(item_name)
                self.current_location.items.remove(item)
                print(f"{item_name} taken.")
                if item_name == "torch":
                    self.solve_blocks("darkness")
                if item_name == "crown":
                    print("You have retrieved the crown and won the game!")
                    self.game_over = True
            else:
                print("You can't take that.")
        elif command == "use torch" and "torch" in self.inventory and "darkness" in [block.name for block in self.current_location.blocks if not block.solved]:
            self.solve_blocks("darkness")
        else:
            print("Unknown command.")

    def solve_blocks(self, block_name):
        block = next((block for block in self.current_location.blocks if block.name == block_name and not block.solved), None)
        if block:
            block.solved = True
            if block_name == "darkness":
                self.locations["royal chamber"].items.append(Item("crown", "The ancient, priceless crown of the kingdom.", can_take=True))
                print(f"{block_name} solved, you can now take the crown.")

if __name__ == "__main__":
    game = Game()
    game.start_game()


You are at the top of the staircase leading down to the entrance hall.
Moving down
You are in the entrance hall of the old castle.
You see a torch: An old but still usable torch.
torch taken.
You are in the entrance hall of the old castle.
Moving north
You are in the grand library, filled with ancient books.
You see a book of lore: This book seems to contain useful information about the castle.
You can't take that.
You are in the grand library, filled with ancient books.
You see a book of lore: This book seems to contain useful information about the castle.
Moving east
This is the royal chamber. The crown must be here somewhere!
You see a crown: The ancient, priceless crown of the kingdom.
darkness solved, you can now take the crown.
This is the royal chamber. The crown must be here somewhere!
You see a crown: The ancient, priceless crown of the kingdom.
You see a crown: The ancient, priceless crown of the kingdom.
crown taken.
You have retrieved the crown and won the game!
